In [6]:
import numpy as np
from numpy import array
import sympy
from sympy import symbols,  cos, sin, pi, simplify, sqrt, atan2, acos, Transpose
from sympy.matrices import Matrix

In [7]:
# Define DH param symbols
q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8') # theta_1
d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8') # d paramater
a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7')
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7')


# Joint angle symbols
# theta1, theta2, theta3, theta4, theta5, theta6 = 0


# Modified DH params
dh = {alpha0:     0, a0:      0, d1:  0.75,
      alpha1: -pi/2, a1:   0.35, d2:     0, q2: q2 - pi/2,
      alpha2:     0, a2:   1.25, d3:     0,
      alpha3: -pi/2, a3: -0.054, d4:  1.50,
      alpha4:  pi/2, a4:      0, d5:     0,
      alpha5: -pi/2, a5:      0, d6:     0,
      alpha6:     0, a6:      0, d7: 0.303, q7: 0}

# General rotations around axes

R_z = Matrix([[       cos(q1),    -sin(q1),             0, 0],
              [       sin(q1),     cos(q1),             0, 0],
              [             0,           0,             1, 0],
              [             0,           0,             0, 1]])
R_y = Matrix([[       cos(q1),           0,       sin(q1), 0],
              [             0,           1,             0, 0],
              [      -sin(q1),           0,       cos(q1), 0],
              [             0,           0,             0, 1]])
R_x = Matrix([[             1,           0,             0, 0],
              [             0,     cos(q1),      -sin(q1), 0],
              [             0,     sin(q1),       cos(q1), 0],
              [             0,           0,             0, 1]])

# Define Modified DH Transformation matrix
R_corr = simplify(R_z.evalf(subs={q1: pi}) * R_y.evalf(subs={q1: -pi/2}))

In [8]:
import math

def dotproduct(v1, v2):
  return sum((a*b) for a, b in zip(v1, v2))

def length(v):
  return math.sqrt(dotproduct(v, v))

def angle(v1, v2):
  return math.acos(dotproduct(v1, v2) / (length(v1) * length(v2)))

# dot with theta2
z_axis = [0, 0, 1]

# dot with theta3
j3_wr = [1.85-0.35, 0.0-0.0, 1.946-2.0]

pos_2 = [0.350, 0.000, 0.750]
pos_3 = [0.350, 0.000, 2.000]
pos_w = [1.733, 0.000, 2.583]

v2_3 = [pos_3[0] - pos_2[0], pos_3[1] - pos_2[1], pos_3[2] - pos_2[2]]
v3_w = [pos_w[0] - pos_3[0], pos_w[1] - pos_3[1], pos_w[2] - pos_3[2]]

print "v2_3: ", v2_3
print "z_axis: ", z_axis
print "j3_wr: ", j3_wr
print "v3_w: ", v3_w
try:   
    theta2 = angle(v2_3, z_axis)
except ValueError:
    theta2 = 0
try:
    theta3 = angle(j3_wr, v3_w)
except ValueError:
    theta3 = 0
    
print "theta2: ", theta2
print "theta3: ", theta3

v2_3:  [0.0, 0.0, 1.25]
z_axis:  [0, 0, 1]
j3_wr:  [1.5, 0.0, -0.05400000000000005]
v3_w:  [1.383, 0.0, 0.5830000000000002]
theta2:  0.0
theta3:  0.434927060211


In [50]:
x,y,z = 1.801, 1.005, 1.148
alpha,beta,gamma = 2.649, 0.683, 1.557 #(y,p,r)
# x, y, z = -0.209973197041, 2.49999627205, 1.60003023044
# alpha, beta, gamma = -0.000442585913517, 0.000232783125596, 0.000765804329612
def find_wrist_center(px, py, pz, roll, pitch, yaw):
    end_effector_length = 0.303
    wx = px - (end_effector_length) * 1.0 * cos(yaw) * cos(pitch)
    wy = py - (end_effector_length) * 1.0 * sin(yaw) * cos(pitch)
    wz = pz - (end_effector_length) * -1.0 * sin(pitch)
    return wx, wy, wz
wx, wy, wz = find_wrist_center(x, y, z, alpha, beta, gamma)

# wx, wy, wz = 1.85000000000000, 0.0, 1.94600000000000
print wx, wy, wz


dist_j3_wr = sqrt((1.85-0.35)*(1.85-0.35) + (1.946-2.0)*(1.946-2.0))

# Testing
# wx, wy, wz = 1.724, 1.526, 1.985
# theta1
theta1 = atan2(wy, wx)

# Finding theta2
# beta1
s1 = sqrt(wx*wx + wy*wy) - dh[a1]
s2 = wz - dh[d1]
beta1 = atan2(s2, s1)

# beta2
s3 = sqrt(s1*s1 + s2*s2)
s4 = sqrt(dh[a3]*dh[a3] + dh[d4]*dh[d4])
beta2_d = (dh[a2]*dh[a2] + s3*s3 - s4*s4) / (2*dh[a2]*s3)
beta2 = atan2(sqrt(1 - beta2_d*beta2_d), beta2_d)

# theta2
theta2 = pi/2 - beta1 - beta2

# Finding theta3
# beta3_intiial
s3_initial = sqrt((1.85-dh[a1])*(1.85-dh[a1])+ (1.946-dh[d1])*(1.946-dh[d1]))
s4_initial = s4
beta4 = pi/2 - atan2(dh[a3], dh[d4])
beta3_initial_d = (dh[a2]*dh[a2] + s4_initial*s4_initial - s3_initial*s3_initial) / (2.0*dh[a2]*s4_initial) 
beta3_initial = atan2(sqrt(1 - beta3_initial_d*beta3_initial_d), beta3_initial_d)
# beta3_initial = 1.5348118667

# beta3
beta3_d = (dh[a2]*dh[a2] + s4*s4 - s3*s3) / (2.0*dh[a2]*s4)
beta3 = atan2(sqrt(1 - beta3_d*beta3_d), beta3_d)

# theta3
theta3 = beta3_initial - beta3
print "Betas"
print "Beta1", beta1.evalf(), "Beta2", beta2.evalf()
print "Beta3", beta3.evalf(), "Beta4", beta4.evalf()
print theta1.evalf(), theta2.evalf(), theta3.evalf()

1.79775752594248 0.769990465910194 1.33923024146008
Betas
Beta1 0.351701920260096 Beta2 1.02077785025322
Beta3 1.33136041189511 Beta4 1.60678078687695
0.404667584932112 0.198316556281581 0.203451454817739
